In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

#빅쿼리 연결

In [ ]:
from google.colab import auth
auth.authenticate_user()

# BigQuery
from google.cloud import bigquery

# viz libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.express as px

# viz settings
%matplotlib inline
sns.set()
init_notebook_mode(connected=True)

import plotly.io as pio
pio.renderers.default = "colab"

!pip install koreanize-matplotlib
import koreanize_matplotlib

%config InlineBackend.figure_format = 'retina'

# Client 객체 생성
client = bigquery.Client(project = '755804531278')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.1 MB/s eta 0:00:00


# [가설 1] 광고의 위치에 따라 클릭여부에 유의미한 차이가 있는가?

In [ ]:
# 광고 위치별 데이터 개수
query = """
SELECT
  fullVisitorId,
  p.promoPosition,
  h.promotionActionInfo.promoIsClick AS promoIsClick
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) AS h,
UNNEST(h.promotion) AS p
WHERE _TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
AND p.promoPosition IS NOT NULL
"""

promo_pos = client.query(query).to_dataframe()
promo_pos['promoIsClick'] = promo_pos['promoIsClick'].fillna(False) #결측치는 False로
promo_pos['promoPosition'].value_counts()

,count
promoPosition,
Brand Row 7-2,631473
Row 1,631385
Brand Row 7-1,631289
Row 5 Color Combo,630195
Row 2 Combo,628741
Row 4 Color Combo,628538
Brand Row 7-3,627189
Row 3-2,626741
Row 3-1,622859


In [ ]:
# [하단 상품 추천 구역에 광고 / 브랜드 전용 구역]의 광고 위치로 재분류
df_brand=promo_pos[promo_pos['promoPosition'].str.contains('Brand')]
df_product=promo_pos[promo_pos['promoPosition'].str.contains('Product')]

# 각각 카운트 집계
brand_count = df_brand.shape[0]
product_count = df_product.shape[0]

# 카운트 결과를 데이터프레임으로 정리
import pandas as pd
count_df = pd.DataFrame({
    'promoPosition': ['Brand', 'Product'],
    'Count': [brand_count, product_count]
})

count_df

,promoPosition,Count
0,Brand,1889951
1,Product,163077


In [ ]:
#클릭여부에 따른 교차표 생성
df = pd.DataFrame({
    'group': ['Brand_position', 'Product_position'],
    'click': [len(df_brand[df_brand['promoIsClick'] == True]), len(df_product[df_product['promoIsClick'] == True])],
    'count': [len(df_brand), len(df_product)]
})

import numpy as np
observed = np.array([
    [df.loc[df['group'] == 'Brand_position', 'click'].values[0], df.loc[df['group'] == 'Brand_position', 'count'].values[0] - df.loc[df['group'] == 'Brand_position', 'click'].values[0]],
    [df.loc[df['group'] == 'Product_position', 'click'].values[0], df.loc[df['group'] == 'Product_position', 'count'].values[0] - df.loc[df['group'] == 'Product_position', 'click'].values[0]]
])
result = pd.DataFrame(observed, index=['Brand_position', 'Product_position'], columns=['click', 'not click'])
result

,click,not click
Brand_position,21374,1868577
Product_position,816,162261


In [ ]:
print('브랜드 전용 구역 광고 클릭률:',21374/1889951)
print('하단 상품 추천 구역 광고 클릭률:',816/163077)

브랜드 전용 구역 광고 클릭률: 0.01130928791275541
하단 상품 추천 구역 광고 클릭률: 0.005003771224636215


In [ ]:
#카이제곱 검정
from scipy.stats import chi2_contingency
chi2_stat, p_value, dof, expected = chi2_contingency(observed)

# 결과 출력
print("===== Chi-Square Test Result =====")
print("Chi2:", chi2_stat)
print("p-value:", p_value)
# print("Expected freq:\n", expected)

===== Chi-Square Test Result =====
Chi2: 557.6837543651368
p-value: 2.6821446406660886e-123


# [가설 2] 광고 유형(adContent)에 따라 전환률에 차이가 있는가?

In [ ]:
#광고 유형(adContent) 전체 확인
query = f"""
SELECT trafficSource.adContent, totals.totalTransactionRevenue
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE _table_suffix between '20160801' and '20170801'
ORDER BY 1
"""
df = client.query(query).result().to_dataframe()
df['totalTransactionRevenue'] = df['totalTransactionRevenue'].astype(float).fillna(0)
df['revenue'] = df['totalTransactionRevenue'] / 1e6

df['adContent'].value_counts()

,count
adContent,
Google Merchandise Collection,5122
Google Online Store,1245
Display Ad created 3/11/14,967
Full auto ad IMAGE ONLY,822
Ad from 12/13/16,610
Ad from 11/3/16,489
Display Ad created 3/11/15,392
{KeyWord:Google Brand Items},251
{KeyWord:Google Merchandise},155


In [ ]:
# 광고 유형(adContent) 재정의 [텍스트 vs 이미지]
ads_text = [
    'Full auto ad TEXT ONLY',
    'Full auto ad TEXT/NATIVE',
   #'Full auto ad NATIVE ONLY'
]
ads_image = ['Full auto ad IMAGE ONLY']

# 대상 광고만 필터링
df_auto = df[df['adContent'].isin(ads_text + ads_image)].copy()

# 광고 타입 라벨링
df_auto['ad_type'] = df_auto['adContent'].apply(lambda x: 'text' if x in ads_text else 'image')

df_auto['ad_type'].value_counts()

,count
ad_type,
image,822
text,23


In [ ]:
# 전환 여부 정의
df_auto['conversion_num'] = (df_auto['revenue'] > 0).astype(int)
df_auto['conversion'] = df_auto['conversion_num'].map({0: '미구매', 1: '구매'})

#교차표 생성
contingency_table = pd.crosstab(df_auto['ad_type'], df_auto['conversion'])
contingency_table

conversion,구매,미구매
ad_type,,
image,5,817
text,0,23


In [ ]:
# 비율 검정
print(f"[전환율 분석]")
successes = df_auto.groupby('ad_type')['conversion_num'].sum().tolist()
nobs = df_auto['ad_type'].value_counts().sort_index().tolist()
print("- text 그룹 구매 전환율:",successes[0]/nobs[0])
print("- image 그룹 구매 전환율:", successes[1]/nobs[1])

# Fisher's exact test 실행
from scipy.stats import fisher_exact
oddsratio, p_val = fisher_exact(contingency_table)
print(f"- Fisher's exact test p-value: {p_val:.5f}")

if p_val < 0.05:
    print("\n 검정 결과 : 전환율 차이가 통계적으로 유의합니다.")
else:
    print("\n 검정 결과 : 전환율 차이가 통계적으로 유의하지 않습니다.")

[전환율 분석]
- text 그룹 구매 전환율: 0.006082725060827251
- image 그룹 구매 전환율: 0.0
- Fisher's exact test p-value: 1.00000

 검정 결과 : 전환율 차이가 통계적으로 유의하지 않습니다.


# [가설 3] 노출 제품수에 따라 이탈률에 차이가 있는가?

In [ ]:
# 데이터 필터링
query = """
WITH impression_data AS (
  SELECT
    fullVisitorId,
    COUNT(p.v2ProductName) AS impression_count,
    SUM(totals.bounces) / SUM(totals.visits) AS exit_rate -- 이탈률
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS h,
    UNNEST(h.product) AS p
  WHERE
    p.isImpression = TRUE
    AND _table_suffix between '20160801' and '20170801'
  GROUP BY
    fullVisitorId
)

SELECT
  fullVisitorId,
  exit_rate,
  CASE
    WHEN impression_count BETWEEN 1 AND 12 THEN '1-12'
    WHEN impression_count > 12 THEN '12+'
    ELSE '0'
  END AS impression_group
FROM
  impression_data

"""

df = client.query(query).to_dataframe()
df= df.dropna(subset=['exit_rate'])
df['impression_group'].value_counts()

,count
impression_group,
1-12,88162
12+,24145


In [ ]:
# 비율 검정
print(f"[이탈률 분석]")

# 두 그룹 이탈률 리스트
group1 = df[df['impression_group'] == '1-12']['exit_rate'].tolist()
group2 = df[df['impression_group'] == '12+']['exit_rate'].tolist()
print("- 노출 제품 수 12개 이하 그룹 이탈률:",sum(group1)/len(group1))
print("- 노출 제품 수 12개 이상 그룹 이탈률:",sum(group2)/len(group2))

# Welch's t-test
from scipy.stats import ttest_ind

stat, p_val = ttest_ind(group1, group2, equal_var=False)
print(f"- t-test p-value: stat={stat:.3f}, p-value={p_val:.5f}")

if p_val < 0.05:
    print("\n검정 결과: 두 노출 수 그룹 간 이탈률 차이가 통계적으로 유의합니다.")
else:
    print("\n검정 결과: 두 노출 수 그룹 간 이탈률 차이가 통계적으로 유의하지 않습니다.")


[이탈률 분석]
- 노출 제품 수 12개 이하 그룹 이탈률: 0.9956279251173876
- 노출 제품 수 12개 이상 그룹 이탈률: 0.46059649124746904
- t-test p-value: stat=220.881, p-value=0.00000

검정 결과: 두 노출 수 그룹 간 이탈률 차이가 통계적으로 유의합니다.
